# Lab 4: Class Imbalance and Performance Metrics

**Exercise 1**

A little classification problem on glass type recognition.

1. Import the `glass.dat` dataset;
2. print the data  and the classes (column `type`);
3. divide the dataset into training and test set with the 20% of the whole data for testing;
4. print the number of training and test instances;
5. train a SVM and compute the accuracy on the test set.

In [ ]:
# YOUR CODE HERE
import pandas as pd
import numpy as np

data = pd.read_csv("glass.dat")
print(f"Number samples: {len(data)}")
print(f"Classes: {data['type'].unique()}")
data.head()

In [ ]:
X = data.to_numpy()[:, :-1]
y = data.values[:, -1]
print(X.shape)
print(y.shape)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"len X_train\t{len(X_train)}")
print(f"len X_test \t{len(X_test)}")

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

# Instantiate the model specifying the kernel
clf = SVC()

# Training
clf.fit(X_train, y_train)

# Prediction
y_pred = clf.predict(X_test)

# Accuracy
print(metrics.accuracy_score(y_test, y_pred))

Are we happy about this accuracy? Let's look the f1-score = harmonic mean of the precision and recall with `metrics.f1_score`

$$
    f1 = 2 \cdot \dfrac{(precision \cdot recall)}{(precision + recall)}
$$

In [ ]:
print(metrics.f1_score(y_pred,y_test))

In [ ]:
print(metrics.recall_score(y_pred,y_test)) # recall = tp / (tp + fn) 
print(metrics.precision_score(y_pred,y_test)) # precision =  tp / (tp + fp)

Both precision and recall are 0, but why? Let's print the entries of the confusion matrix with `metrics.confusion_matrix`.


<img src="img/conf_mat.jpg" alt="Drawing" style="width: 500px;"/>

In [ ]:
conf_mat = metrics.confusion_matrix(y_test, y_pred)
print(type(conf_mat))
print("confusion matrix \n",conf_mat,"\n")
print("True negative    \t",conf_mat[0][0],"--> predicted as negative, and really negative :) ")
print("False positive   \t",conf_mat[0][1]," --> predicted as positive, but negative :( ")
print("False negative  \t",conf_mat[1][0]," --> predicted as negative, but positive :( ")
print("True positive   \t",conf_mat[1][1]," --> predicted as positive, and really positive :) ")

In [ ]:
# have a look at the predictions:

print(y_pred)

The classifier predicts only one classe (the class zero), that's really bad! Any idea?

**Exercise 2**

1. Print the number of instances of class 0 and 1. Hint: you can use `np.unique` by setting a proper input parameter, have a look at the documentation.
2. Print the ratio of positive samples over the total samples -> the skewness.
3. Print the ratio of negative samples over the total samples.
4. Is the dataset balanced?

In [ ]:
# YOUR CODE HERE
print(np.unique(y_train, return_counts=True))
# the dataset is not balanced

In [ ]:
# YOUR CODE HERE

neg = np.unique(y_train, return_counts=True)[1][0]
pos = np.unique(y_train, return_counts=True)[1][1]

print(f"Number of pos instances\t{pos}")
print(f"Number of neg instances\t{neg}")
print(f"Positive ratio\t{pos/(pos+neg)}")
print(f"Negative ratio\t{neg/(pos+neg)}")

It is a good practice to compute the positive and negative ratio before training a ML to understand the imbalance of the data.

In the next exercise we address the imbalance by oversampling the minority class: giving more weights to the minority class corresponds to duplicate the samples of the minority class to reach the same number of the majority class.

**Exercise 3**
1. Train a SVM by including the class weights (see the doc, use both the `class_weight` option);
2. The `class_weight` option takes a dict as `{0: weight_class_0, 1: weight_class_1}`, we can give weight 1 to the most popular class and a bigger weight to the minority class.
3. Print precision, recall, f1 score, accuracy and confusion matrix.

In [ ]:
from sklearn.svm import SVC
print(f"Neg weights: \n{neg/pos}")
# Instantiate the model specifying the kernel
clf = SVC(class_weight={0: 1, 1: neg/pos})

# Training
clf.fit(X_train, y_train)

# Prediction
y_pred = clf.predict(X_test)

# Performance
print(f"\nPrecision: \n{metrics.precision_score(y_test,y_pred)}")
print(f"\nRecall: \n{metrics.recall_score(y_test,y_pred)}")
print(f"\nF1: \n{metrics.f1_score(y_test,y_pred)}")
print(f"\nAccuracy: \n{metrics.accuracy_score(y_test,y_pred)}")
conf_mat = metrics.confusion_matrix(y_test,y_pred)
print(f"\nConfusion Matrix:\n{conf_mat}")

## Plot Receiver Operating Characteristic and Precision/Recall curves

Sklearn provides you utilities for computing the ROC and the PrecRec curve: the `roc_curve` and `precision_recall_curve` functions in `sklearn.metrics`. In addition, the `auc` function returns the area under each curve.

**Exercise 4**
Plot the ROC curve for each class of the Iris dataset. Understand and complete the following code.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier


# Import the IRIS dataset by using the datasets.load_iris() function and inspect its attributes
# YOUR CODE HERE
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binarize the output
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(10)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets: test size 50%, random_state=0
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel="linear", probability=True, random_state=random_state))

# Train the classifier and compute the scores of the test set (look at the SVM methods)
# YOUR CODE HERE
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
# Understand what the roc_curve() does and takes as input
fpr = dict()
tpr = dict()
th = dict()
roc_auc = dict()
for i in range(n_classes):
    # YOUR CODE HERE
    fpr[i], tpr[i], th[i] = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Plot the ROC curve for each class with the AUC in the legend. Plot also the line from (0, 0) to (1, 1)
# YOUR CODE HERE
plt.figure(figsize=(7, 7))
lw = 2

colors = ["aqua", "darkorange", "cornflowerblue"]

for class_id, class_name in enumerate(iris.target_names):
    plt.plot(fpr[class_id], tpr[class_id], lw=lw, c=colors[class_id], label=f"ROC curve for {class_name} (AUC = {roc_auc[class_id]:.2f})")

plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic\nto One-vs-Rest multiclass")
plt.legend(loc="lower right")

This task can be performed with the `RocCurveDisplay` function in `sklearn.metrics`. The boolean parameter `plot_chance_level` (new from version 1.3.) determines whether to plot the baseline level.

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

colors = ["aqua", "darkorange", "cornflowerblue"]
fig, ax = plt.subplots(figsize=(7, 7))

for class_id, class_name in enumerate(iris.target_names):
    RocCurveDisplay.from_predictions(
        y_test[:, class_id],
        y_score[:, class_id],
        name=f"ROC curve for {class_name}",
        color=colors[class_id],
        ax=ax)
    
_ = ax.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="Receiver Operating Characteristic\nto One-vs-Rest multiclass")

**Exercise 5**

Repeat the same exercise by plotting the Precision Recall curve (search for the `precision_recall_curve` SKlearn function) with the baseline: P/(P + N)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

# Import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binarize the output
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets: test size 50%, random_state=0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel="linear", probability=True, random_state=random_state))

# Train the classifier and compute the scores of the test set (look at the SVM methods)
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

precision = dict()
recall = dict()
pr_auc = dict()
th = dict()
for i in range(n_classes):
    precision[i], recall[i], th[i] = precision_recall_curve(y_test[:, i], y_score[:, i])
    pr_auc[i] = auc(recall[i], precision[i])

plt.figure(figsize=(7, 7))
lw = 2
colors = ["aqua", "darkorange", "cornflowerblue"]

for class_id, class_name in enumerate(iris.target_names):
    plt.plot(recall[class_id], precision[class_id], c=colors[class_id], lw=lw, label=f"Precision-recall for {class_name} (AUC = {pr_auc[class_id]:.2f})")

classes, occurences = np.unique(iris.target, return_counts=True)
skew = occurences/np.sum(occurences)

for class_id, class_name in enumerate(iris.target_names):
    plt.plot([0, 1], [skew[class_id], skew[class_id]], color="navy", lw=lw, linestyle="--", label=f'Baseline for {class_name}')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision Recall curve example")
plt.legend(loc="lower right")

This task can be performed with the `PrecisionRecallDisplay` function in `sklearn.metrics`.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score, precision_recall_curve, PrecisionRecallDisplay

precision = dict()
recall = dict()
average_precision = dict()

fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title="Precision Recall Curve\nto One-vs-Rest multiclass",

colors = ["aqua", "darkorange", "cornflowerblue"]
for class_id, class_name in enumerate(iris.target_names):
    precision[class_id], recall[class_id], _ = precision_recall_curve(y_test[:, class_id], y_score[:, class_id])
    average_precision[class_id] = average_precision_score(y_test[:, class_id], y_score[:, class_id])
    display = PrecisionRecallDisplay(
        recall=recall[class_id],
        precision=precision[class_id],
        average_precision=average_precision[class_id],
    )
    display.plot(ax=ax, name=f"Precision-recall for class {class_name}", color=colors[class_id])

plt.show()

## Model Selection: K fold, cross validation

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called **overfitting**. To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set `X_test`, `y_test`. Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally.

One problem with validation sets is that you "lose" some of the data: you only used, e.g., 3/4 of the data for the training, and used 1/4 for the validation. One option is to use K-fold cross-validation, where we split the data into chunks and perform fits, where each chunk gets a turn as the validation set. 

First, we need to understand how to split data into folds by using `KFold`:

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

data = pd.read_csv("glass.dat")
X = data.iloc[:, :-1].values
y = data['type'].values

kf = KFold(n_splits=3, shuffle=True, random_state=0)

for fold_id,  (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold id {fold_id} TRAIN: {train_index}")
    print(f"Fold id {fold_id} TEST: {test_index}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(f"Label counts: {np.unique(y_train, return_counts=True)}")
    print("--------------------------------------------------------------------------------\n")

We can notice that some y labels are not balanced in the folds, use `StratifiedKFold` to obtain folds with balanced labels.

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

for fold_id,  (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold id {fold_id} TRAIN: {train_index}")
    print(f"Fold id {fold_id} TEST: {test_index}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(f"Label counts: {np.unique(y_train, return_counts=True)}")
    print("--------------------------------------------------------------------------------\n")





When evaluating different settings (“hyperparameters”) for estimators, such as the C setting that must be manually set for an SVM, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets. The following procedure is followed for each of the k “folds”:

- A model is trained using $K - 1$ of the folds as training data;

- the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as is the case when fixing an arbitrary validation set), which is a major advantage in problems such as inverse inference where the number of samples is very small.

<br><br>
<img src="img/grid_search_cross_validation.png" style="width: 400px;"/>
<br><br>

Here is a flowchart of typical cross validation workflow in model training. The best parameters can be determined by grid search techniques:

<br><br>
<img src="img/grid_search_workflow.png" style="width: 400px;"/>
<br><br>

**Exercise 6**

Use the `google_play_store_apps_reviews_training` dataset to perform 5-fold cross-validation:

- Look at Lab 3 for the data import, preprocessing and splitting into train and test. Use TfidfVectorizer without stopwords;
- Combine a `StratifiedKFold` with with `cross_val_score` of `sklearn.model_selection` to train a Support Vector Machine with linear kernel. As classification score in the `cross_val_score` object use the f1 measure (see [here](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)) in the `scoring` parameter (see [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)). For
- print the mean and stdev of the cross validation scores.

In [ ]:
### YOUR CODE HERE
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

data = pd.read_csv('google_play_store_apps_reviews_training.csv')
vec = TfidfVectorizer()
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# data cleaning
data = data.drop('package_name', axis=1)
data['review'] = data['review'].str.strip().str.lower().str.replace('[^\w]', ' ')

# Split into training and testing data
X = data['review']
y = data['polarity']

X = vec.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf = svm.SVC(kernel='linear', C=1)

scores = cross_val_score(clf, X_train, y_train, cv=kf, scoring='f1')
print(scores)
print(scores.mean(), scores.std())

**Exercise 7**

You can create a Sklearn pipeline to embed several ML steps. Pipelines can be used as normal classifiers, here an example:

In [ ]:
from sklearn.pipeline import make_pipeline

data = pd.read_csv('google_play_store_apps_reviews_training.csv')

# data cleaning
data = data.drop('package_name', axis=1)
data['review'] = data['review'].str.strip().str.lower().str.replace('[^\w]', ' ')

# Split into training and testing data
X = data['review']
y = data['polarity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

clf_pipeline = make_pipeline(CountVectorizer(), svm.SVC(kernel='linear', C=1))
scores = cross_val_score(clf_pipeline, X_train, y_train, cv=kf, scoring='f1')
print(scores.mean(), scores.std())

Implement the following 3 pipelines for the above classification and evaluate them with cross_val_score:
- `TfidfVectorizer()` and `svm.SVC(kernel='linear', C=1)`;
- `CountVectorizer()` and the feature `Normalizer()` step, `svm.SVC(kernel='linear', C=1)`;
- `CountVectorizer()` and `svm.SVC(kernel='linear', C=1)`.

In [ ]:
### YOUR CODE HERE
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn import svm

data = pd.read_csv('google_play_store_apps_reviews_training.csv')

# data cleaning
data = data.drop('package_name', axis=1)
data['review'] = data['review'].str.strip().str.lower().str.replace('[^\w]', ' ')

# Split into training and testing data
X = data['review']
y = data['polarity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

pipe_1 = make_pipeline(TfidfVectorizer(), svm.SVC(kernel='linear', C=1))
pipe_2 = make_pipeline(CountVectorizer(), preprocessing.Normalizer(), svm.SVC(kernel='linear', C=1))
pipe_3 = make_pipeline(CountVectorizer(), svm.SVC(kernel='linear', C=1))

pipe_list = [pipe_1, pipe_2, pipe_3]
for pipe in pipe_list:
    print(pipe)
    scores = cross_val_score(pipe, X_train, y_train, cv=kf, scoring='f1')
    print(scores.mean(), scores.std())
    print("--------------------------------------------------------------------------------\n")

**Exercise 8**

Repeat exercise 7 by using `GridSearchCV` to perform an exhaustive search over a set of hyperparameters for a SVM with 'rbf' kernel:
- user `TfidfVectorizer` as vectorizer;
- use `StratifiedKFold` with 5 folds;
- you have to define a hyperparameters grid with a Python dict: the keys are the hyperparam name, the values the range of values for that hyperparam;
- use `[1, 10, 100]` for C and `[0.01, 0.05, 0.1]` for gamma;
- perform `GridSearchCV`: use the f1 for the score and try to run the fold in parallel:look at the documentation how it is used;
- export the results in a Pandas dataframe and print it.
- What is the hyperparameters combination leading to the best average f1? What is the value of the best average f1?

In [9]:
### YOUR CODE HERE
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

data = pd.read_csv('google_play_store_apps_reviews_training.csv')
vec = TfidfVectorizer()

# data cleaning
data = data.drop('package_name', axis=1)
data['review'] = data['review'].str.strip().str.lower().str.replace('[^\w]', ' ')

# Split into training and testing data
X = data['review']
y = data['polarity']

X = vec.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Set up possible values of parameters with a dict (a parameter grid) to optimize over.
# Use [1, 10, 100] for C and [0.01, 0.05, 0.1] for gamma.
p_grid = {"C": [1, 10, 100], "gamma": [0.01, 0.05, 0.1]}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
svc = svm.SVC()

# Perform GridSearchCV: use the f1 for the score and try to run the fold in parallel.
estimator = GridSearchCV(svc, p_grid, cv=kf, scoring='f1', n_jobs=-1)
estimator.fit(X_train, y_train)

# Export the results in a Pandas dataframe and print it.
# What is the hyperparameters combination leading to the best average f1? What is the value of the best average f1?
res_grid_search = pd.DataFrame(estimator.cv_results_)
print(estimator.best_params_)
print(estimator.best_score_)
print(res_grid_search)

# Lest compare the SVM with and without parameter optimization
# svc = svm.SVC(C=100, gamma=0.05)
svc = svm.SVC(**estimator.best_params_)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(f1_score(y_test, y_pred))

svc = svm.SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(f1_score(y_test, y_pred))

{'C': 100, 'gamma': 0.05}
0.730544411027281
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       0.093242      0.002879         0.030210        0.008084       1   
1       0.087106      0.009190         0.026903        0.006001       1   
2       0.085982      0.003986         0.026619        0.002381       1   
3       0.087308      0.004251         0.022393        0.003034      10   
4       0.086498      0.008681         0.023769        0.005037      10   
5       0.098306      0.010285         0.021538        0.005038      10   
6       0.098951      0.004908         0.022549        0.005285     100   
7       0.101376      0.005831         0.017250        0.001975     100   
8       0.091850      0.005800         0.017036        0.001346     100   

  param_gamma                     params  split0_test_score  \
0        0.01    {'C': 1, 'gamma': 0.01}           0.000000   
1        0.05    {'C': 1, 'gamma': 0.05}           0.000000   
2         0.1   